In [1]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

import numpy as np



In [2]:
# ИРИСЫ ИРИСЫ ИРИСЫ
iris = load_iris()

X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.2, random_state=42)


In [3]:
# дерево обычное сайкитлёрновское
tree = DecisionTreeClassifier(criterion='gini', splitter='best', max_depth=None, min_samples_leaf=1, min_samples_split=2, max_features=None)
tree.fit(X_train, y_train)


DecisionTreeClassifier()

In [4]:
y_pred = tree.predict(X_test)

In [5]:
# Оценка 
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

Accuracy: 1.0


In [6]:
#СДЕЛАЙ САМ

class MyDecisionTree:
    def __init__(self, max_depth=None, min_samples_leaf=1):
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.tree = None

    def gini(self, y):
        classes, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        gini_impurity = 1 - np.sum(probabilities ** 2)
        return gini_impurity

    def split(self, X, y, feature_index, threshold):
        mask = X[:, feature_index] <= threshold
        return X[mask], y[mask], X[~mask], y[~mask]

    def find_best_split(self, X, y):
        m, n = X.shape
        if m <= 1:
            return None, None

        current_gini = self.gini(y)
        best_gini = 1
        best_feature = None
        best_threshold = None

        for feature_index in range(n):
            thresholds = np.unique(X[:, feature_index])
            for threshold in thresholds:
                X_left, y_left, X_right, y_right = self.split(X, y, feature_index, threshold)

                if len(y_left) < self.min_samples_leaf or len(y_right) < self.min_samples_leaf:
                    continue

                gini_left = self.gini(y_left)
                gini_right = self.gini(y_right)

                weighted_gini = (len(y_left) / m) * gini_left + (len(y_right) / m) * gini_right

                if weighted_gini < best_gini:
                    best_gini = weighted_gini
                    best_feature = feature_index
                    best_threshold = threshold

        return best_feature, best_threshold

    def build_tree(self, X, y, depth):
        if depth == 0 or len(np.unique(y)) == 1:
            return np.argmax(np.bincount(y))

        best_feature, best_threshold = self.find_best_split(X, y)

        if best_feature is None:
            return np.argmax(np.bincount(y))

        X_left, y_left, X_right, y_right = self.split(X, y, best_feature, best_threshold)

        left_subtree = self.build_tree(X_left, y_left, depth - 1)
        right_subtree = self.build_tree(X_right, y_right, depth - 1)

        return (best_feature, best_threshold, left_subtree, right_subtree)

    def fit(self, X, y):
        self.tree = self.build_tree(X, y, self.max_depth)

    def predict_instance(self, x, node):
        if np.isscalar(node):
            return node  # leaf node

        feature, threshold, left_subtree, right_subtree = node
        if x[feature] <= threshold:
            return self.predict_instance(x, left_subtree)
        else:
            return self.predict_instance(x, right_subtree)

    def predict(self, X):
        return np.array([self.predict_instance(x, self.tree) for x in X])
    


In [7]:
# Дерево необычное, не сайкитлёрновское
tree = MyDecisionTree(max_depth=3, min_samples_leaf=1)
tree.fit(X_train, y_train)

In [8]:
y_pred = tree.predict(X_test)

In [9]:
# Оценка 
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

Accuracy: 1.0
